In [12]:
import sagemaker
print(sagemaker.__file__)


c:\Users\MY\anaconda3\envs\sagemaker\lib\site-packages\sagemaker\__init__.py


In [13]:
import boto3
import pandas as pd

region = boto3.session.Session().region_name
bucket = "manubucketsagemaker"

print("Using bucket:", bucket)
print("Region:", region)


Using bucket: manubucketsagemaker
Region: us-east-1


In [14]:
df = pd.read_csv("train.csv")

In [15]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [16]:
df.shape

(2000, 21)

In [17]:
df["price_range"].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [18]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [19]:
df.shape

(2000, 21)

In [20]:
df.isnull().mean()*100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [21]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [22]:
label = features.pop(-1)
label

'price_range'

In [23]:
x = df[features]
y = df[label]

In [24]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [25]:
y.head()
# {0: "Low risk"
#  1: "High risk"}

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [26]:
x.shape

(2000, 20)

In [27]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= 0.15, random_state=0)

In [30]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [31]:
trainX = pd.DataFrame(X_train)
trainX[label]= y_train
testX = pd.DataFrame(X_test)
testX[label]= y_test

In [32]:
print(trainX.shape)
print(testX.shape)

(1700, 21)
(300, 21)


In [33]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0


In [34]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [35]:
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [36]:
trainX.to_csv("train-V-1.csv",index=False)
testX.to_csv("test-V-1.csv",index=False)

In [37]:
import boto3

s3_client = boto3.client("s3", region_name=region)

sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"

train_file = "train-V-1.csv"
test_file = "test-V-1.csv"

train_s3_path = f"s3://{bucket}/{sk_prefix}/train-V-1.csv"
test_s3_path = f"s3://{bucket}/{sk_prefix}/test-V-1.csv"

s3_client.upload_file(train_file, bucket, f"{sk_prefix}/train-V-1.csv")
s3_client.upload_file(test_file, bucket, f"{sk_prefix}/test-V-1.csv")

print("Train:", train_s3_path)
print("Test:", test_s3_path)



Train: s3://manubucketsagemaker/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
Test: s3://manubucketsagemaker/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


In [38]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import sklearn
import joblib
import argparse
import os
import pandas as pd

# Required by SageMaker for inference
def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model


if __name__ == "__main__":
    print("[INFO] Extracting arguments")

    parser = argparse.ArgumentParser()

    # Hyperparameters
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # SageMaker paths
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))

    # File names
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()

    print("SKLearn Version:", sklearn.__version__)
    print("Joblib Version:", joblib.__version__)

    print("\n[INFO] Reading data")

    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    X_train = train_df[features]
    y_train = train_df[label]

    X_test = test_df[features]
    y_test = test_df[label]

    print("\nColumn order:")
    print(features)

    print("\nLabel column:", label)

    print("\nTraining data shape:", X_train.shape)
    print("Testing data shape:", X_test.shape)

    print("\nTraining RandomForest model...")

    model = RandomForestClassifier(
        n_estimators=args.n_estimators,
        random_state=args.random_state
    )

    model.fit(X_train, y_train)

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)

    print("Model persisted at:", model_path)

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)

    print("\n--- TEST METRICS ---")
    print("Accuracy:", test_acc)
    print("Classification Report:")
    print(classification_report(y_test, y_pred_test))



Overwriting script.py


In [42]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import image_uris

sess = sagemaker.Session()
region = sess.boto_session.region_name

image_uri = image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="1.2-1",
    py_version="py3",
    instance_type="ml.m5.large",
)

sklearn_estimator = Estimator(
    image_uri=image_uri,
    entry_point="script.py",
    role="arn:aws:iam::029280391768:role/AmazonSageMakerExecutionRole",
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="rf-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    sagemaker_session=sess,
)

sklearn_estimator.fit(
    inputs={"train": train_s3_path, "test": test_s3_path},
    wait=True
)



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\MY\AppData\Local\sagemaker\sagemaker\config.yaml
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: rf-custom-sklearn-2026-01-02-08-33-22-559


2026-01-02 08:33:29 Starting - Starting the training job...
2026-01-02 08:34:05 Downloading - Downloading input data...
2026-01-02 08:34:30 Downloading - Downloading the training image...../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2026-01-02 08:35:30,588 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2026-01-02 08:35:30,592 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2026-01-02 08:35:30,594 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2026-01-02 08:35:30,610 sagemaker_sklearn_container.training INFO     Invoking user training script.
2026-01-02 08:35:30,871 sagemaker-train

In [43]:
sklearn_estimator.fit(
    {"train": train_s3_path, "test": test_s3_path},
    wait=True
)



Using provided s3_resource


INFO:sagemaker:Creating training-job with name: rf-custom-sklearn-2026-01-02-08-36-30-870


2026-01-02 08:36:33 Starting - Starting the training job...
2026-01-02 08:36:52 Starting - Preparing the instances for training...
2026-01-02 08:37:44 Downloading - Downloading the training image......
2026-01-02 08:38:53 Training - Training image download completed. Training in progress.
2026-01-02 08:38:53 Uploading - Uploading generated training model/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2026-01-02 08:38:47,084 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2026-01-02 08:38:47,090 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2026-01-02 08:38:47,094 sagemaker-training-toolkit INFO     No Neurons detected (norma

In [48]:
import boto3

# use the SAME region where your training job ran
region = "us-east-1"   # change if your job used a different region

sm_boto3 = boto3.client("sagemaker", region_name=region)


In [49]:
# Wait for training job to finish
sklearn_estimator.latest_training_job.wait(logs="None")

# Get S3 model artifact path
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifacts persisted at:", artifact)




2026-01-02 08:39:06 Starting - Preparing the instances for training
2026-01-02 08:39:06 Downloading - Downloading the training image
2026-01-02 08:39:06 Training - Training image download completed. Training in progress.
2026-01-02 08:39:06 Uploading - Uploading generated training model
2026-01-02 08:39:06 Completed - Training job completed
Model artifacts persisted at: s3://sagemaker-us-east-1-029280391768/rf-custom-sklearn-2026-01-02-08-36-30-870/output/model.tar.gz


In [51]:
artifact

's3://sagemaker-us-east-1-029280391768/rf-custom-sklearn-2026-01-02-08-36-30-870/output/model.tar.gz'

In [56]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

FRAMEWORK_VERSION = "1.2-1"   # ✅ ADD THIS

model_name = "custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::029280391768:role/AmazonSageMakerExecutionRole",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\MY\AppData\Local\sagemaker\sagemaker\config.yaml


In [57]:
endpoint_name = "Custom-sklearn-model-"+ strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2026-01-02-09-50-33
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\MY\AppData\Local\sagemaker\sagemaker\config.yaml


INFO:sagemaker:Creating model with name: custom-sklearn-model-2026-01-02-09-44-50
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2026-01-02-09-50-33
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2026-01-02-09-50-33


------!